In [1]:
import numpy as np

Argmax

In [ ]:
#argmax is defined as the index of the maximum value in a given array
def Argmax(x):
    y = max(x)
    return x.index(y)

x = [1, 5, 4, 3]
print(Argmax(x))

1


Mean

In [7]:
#mean is just sum of all elements divided by number of elements
def Mean(x):
    total = 0
    for i in range(len(x)):
        total += x[i]
    mean = total/len(x)
    return mean

print(Mean(x))

3.25


Softmax

In [18]:
x = [1001, 1002, 1003]

In [19]:
#softmax formula is used to turn raw numbers into probability scores that sum up to one
#using the same array as above
def Softmax(x):
    soft = []
    exp = []
    for i in range(len(x)):
        z = x[i]
        exp.append(np.exp(z))

    s = np.sum(exp)
    for i in range(len(x)):
        z = x[i]
        soft.append(np.exp(z)/s)
    return soft

print(Softmax(x))

[nan, nan, nan]


/var/folders/dr/7jrw49755h70bsy03x6hy14c0000gn/T/ipykernel_22462/2029364812.py:8: RuntimeWarning: overflow encountered in exp
  exp.append(np.exp(z))
/var/folders/dr/7jrw49755h70bsy03x6hy14c0000gn/T/ipykernel_22462/2029364812.py:13: RuntimeWarning: overflow encountered in exp
  soft.append(np.exp(z)/s)
/var/folders/dr/7jrw49755h70bsy03x6hy14c0000gn/T/ipykernel_22462/2029364812.py:13: RuntimeWarning: invalid value encountered in scalar divide
  soft.append(np.exp(z)/s)


softmax with numerical stability

In [20]:
#same thing as above, but subtbract the max from each element for numerical stability
def Softmax_stable(x):
    soft = []
    exp = []
    maax = max(x)
    for i in range(len(x)):
        z = x[i]
        exp.append(np.exp(z - maax))

    s = np.sum(exp)
    for i in range(len(x)):
        z = exp[i]
        soft.append(z/s)
    return soft

print(Softmax_stable(x))

[0.09003057317038046, 0.24472847105479764, 0.6652409557748218]


this is identical to

In [21]:
import torch
x = torch.tensor([1001, 1002, 1003])
torch.nn.functional.softmax(x - x.max(), dim=-1)


ModuleNotFoundError: No module named 'torch'